In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2000000)

The purpose of this is to display and understand why the order of the where filters matter when you index spefically with a range involved.
So in this case we will filter on the price and category id. This could be arbitary just as an example.

Initialy i would have the assumtion if we didn't have any index then the fast query would be the one where we only do where category_id = 5. Because i would assume that this would be easier to find because it does not have to do any computation to check if the price is between a range

In [ ]:
query = """
    SELECT name, price FROM products WHERE price BETWEEN 100 AND 200;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

In [ ]:
query = """
    SELECT name, price FROM products WHERE category_id = 5;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

In [ ]:
query = """SELECT name, price FROM products WHERE price BETWEEN 100 AND 200 and category_id = 5;"""
execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)


What i found out was the the latter where you include both where price between and cateogory_id = is the fastest. And this did not initially make sense to me because i thought that the between would be more costly. 

What i know think is the case is that the acutal search is very small and what ends up making this faster is that the amount of data to fetch/blocks to read is the actual bottle neck and there by making it faster

_______

Lets run an explain on all of them

In [ ]:
query = """
    EXPLAIN SELECT name, price FROM products WHERE price BETWEEN 100 AND 200;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
query = """
    EXPLAIN SELECT name, price FROM products WHERE category_id = 5;
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=True)

In [ ]:
query = """EXPLAIN SELECT name, price FROM products WHERE price BETWEEN 100 AND 200 and category_id = 5;"""
execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)


From this explain i wondered why the type was ref for the other two. and that had to do with the foregin key. Because eventhough i have not created a index myself for this tabe because the table is has a primary key i think automatically MySQl will reuse this index or something like that 

In [ ]:
show_table_indexes("products", get_database_name())

Would this all look different if no foregin key is involved?

In [ ]:
query = """
    SELECT name FROM products WHERE price BETWEEN 100 AND 200;
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
query = """
    SELECT name FROM products WHERE size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


In [ ]:
query = """
    SELECT name, price FROM products WHERE price BETWEEN 100 AND 200 AND size = 'Large';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)


This time they are all full scan and my initial assumption seems to acutally hold true.

Lets when moving forward stick to using the Size instead of category id

This time lets create a index for both of the columns and run the same test

In [ ]:
query = "CREATE INDEX idx_price ON products (price)"

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

query = "CREATE INDEX idx_size ON products (size)"
execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)

In [ ]:
execute_query("SELECT name, price, size FROM products where price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=True, show_metrics=False)


execute_query("SELECT name, price, size FROM products where size = 'Large'", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)


execute_query("SELECT name, price, size FROM products where price BETWEEN 100 AND 200 and size = 'Large'", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where price BETWEEN 100 AND 200 and size = 'Large'", database=get_database_name(), print_as_df=True, show_metrics=False)


execute_query("SELECT name, price, size FROM products where size = 'Large' and price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN SELECT name, price, size FROM products where size = 'Large' and price BETWEEN 100 AND 200", database=get_database_name(), print_as_df=True, show_metrics=False)

I read this article: https://use-the-index-luke.com/sql/where-clause/searching-for-ranges/greater-less-between-tuning-sql-access-filter-predicates

and from this i expected there to be a difference between the last two quries but there isn't because it seems that mysql pick the right key so that is a bit curious i think. so just to really test if there is a difference lets try to drop the index on the size

In [ ]:
execute_query("DROP index idx_size ON products", database=get_database_name(), print_as_df=False, show_metrics=True)

In [ ]:
clear_mysql_cache()

query = "SELECT name, price, size FROM products where price BETWEEN 100 AND 110"

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

In [ ]:
clear_mysql_cache()

query = "SELECT name, price, size FROM products where price BETWEEN 100 AND 200"

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " + query, database=get_database_name(), print_as_df=True, show_metrics=False)

This is something fun and Quirky it seems that the MySql execution optimizer will weight that because it guesses that a high amount of the rows will match the query then it will use a full table scan. But if it things that not that many rows will match the filter then it will use the key. 